-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [281]:
#import libraries
import pandas as pd
import numpy as np
import librosa 
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
# from keras.utils import to_categorical

Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [282]:
def extract_features(filename):
    y, sr = librosa.load(filename)
    features = {}

    MFCC = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    melspec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    # melspec_dB = librosa.power_to_db(melspec, ref=np.max)
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

    features['mfcc'] = np.mean(MFCC, axis=1)
    features['chroma'] = np.mean(chroma, axis=1)
    features['melspectrogram'] = np.mean(melspec, axis=1)
    features['contrast'] = np.mean(contrast, axis=1)

    stacked_features = np.hstack([features['mfcc'], features['chroma'], features['melspectrogram'], features['contrast']])

    return stacked_features
    # return features
    pass

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [284]:
def load_data(test_size):
    audios = [os.path.join(os.getcwd(), "Audio_Speech_Actors"), os.path.join(os.getcwd(), "Audio_Song_Actors")]
    features = []
    emotions =[]

    for audio in audios:
        for actor in os.listdir(audio):
            actor_path = os.path.join(audio, actor)
            for audio_file in os.listdir(actor_path):
                file_path = os.path.join(actor_path, audio_file)
                feature = extract_features(file_path)
                # print(f"Feature shape extracted from {file_path}: {feature['mfcc'].shape}, {feature['chroma'].shape}, {feature['melspectrogram'].shape}")
                
                one_hot = np.zeros(8)
                emotion_code = int(audio_file.split("-")[2])
                one_hot[emotion_code-1] = 1
                features.append(feature)
                emotions.append(one_hot)

    X = np.array(features)
    y = np.array(emotions)

    # print("Total features and emotions collected:", len(X), len(y))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    print("Training data shape:", X_train.shape)
    return X_train, X_test, y_train, y_test                

    pass

Use the load_data function to load data and split the data using the train_test_split function.

In [285]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, y_train, y_test = load_data(0.2)

Training data shape: (1961, 187)


Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [287]:
#define the model.
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from keras.regularizers import l2
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(300, input_shape=(X_train.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# model.add(Dense(250, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# model.add(Dense(150, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))


optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'] )
model.summary()

# callbacks = [
#     # EarlyStopping(monitor='val_loss', patience=0, restore_best_weights=True),  # Stop training if no improvement for 10 epochs
#     ModelCheckpoint(filepath='best_model.keras', monitor='val_accuracy', save_best_only=True),  # Save the best model based on validation accuracy
#     # ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)  # Reduce learning rate if validation loss plateaus
# ]

checkpoint_callback = ModelCheckpoint(filepath='best_model.keras', monitor='val_accuracy', save_best_only=True)

def lr_scheduler(epoch):
    if epoch < 50:
        return 0.001
    elif epoch < 100:
        return 0.0005
    else:
        return 0.0001

lr_schedule = LearningRateScheduler(lr_scheduler)


Model: "sequential_79"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_237 (Dense)               │ (None, 300)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_132         │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_109 (Dropout)           │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_238 (Dense)               │ (None, 200)            │        60,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_133         │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_110 (Dropout)           │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_239 (Dense)               │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_134         │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_111 (Dropout)           │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_240 (Dense)               │ (None, 8)              │           808 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,908 (546.52 KB)

 Trainable params: 138,708 (541.83 KB)

 Non-trainable params: 1,200 (4.69 KB)

Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [288]:
#fit the model on training data with validation split of 0.2
model.fit(X_train, y_train, batch_size=256, epochs=1000, validation_split=0.2, callbacks=[lr_schedule, checkpoint_callback])

Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.1561 - loss: 3.0608 - val_accuracy: 0.0738 - val_loss: 21.3906 - learning_rate: 0.0010
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1730 - loss: 2.7007 - val_accuracy: 0.1578 - val_loss: 21.7773 - learning_rate: 0.0010
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2085 - loss: 2.5009 - val_accuracy: 0.1578 - val_loss: 20.9149 - learning_rate: 0.0010
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2254 - loss: 2.4132 - val_accuracy: 0.1578 - val_loss: 18.6024 - learning_rate: 0.0010
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2548 - loss: 2.3273 - val_accuracy: 0.1578 - val_loss: 16.4538 - learning_rate: 0.0010
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2558 - loss: 2.2790 - val_accuracy: 0.1578 - val_loss: 14.0787 - learning_rate: 0.0010
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2678 - loss: 2.1790 - v

In [289]:
#predict for testing data and printout the accuracy
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
percent = accuracy*100
print(f"Accuracy: {percent:.2f}%")

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Accuracy: 71.08%


Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [290]:
#print the classification report for the model
from sklearn.metrics import classification_report
report = classification_report(np.argmax(y_test, axis = 1), np.argmax(y_pred, axis = 1))
print(report)

              precision    recall  f1-score   support

           0       0.61      0.67      0.64        33
           1       0.72      0.75      0.73        75
           2       0.72      0.68      0.70        76
           3       0.66      0.63      0.65        82
           4       0.83      0.85      0.84        62
           5       0.75      0.65      0.70        89
           6       0.60      0.64      0.62        33
           7       0.70      0.85      0.77        41

    accuracy                           0.71       491
   macro avg       0.70      0.72      0.71       491
weighted avg       0.71      0.71      0.71       491



Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)